In [238]:

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# ONE LINE

In [239]:
model = tf.keras.models.load_model('detectorV6.h5')

In [240]:
testDataRaw = pd.read_csv('../../../datasets/fraudTest.csv')

In [278]:
# Seleccionar una línea aleatoria del dataset fraudTest.csv
random_line = pd.read_csv('../../../datasets/fraudTest.csv').sample(1)
random_fraud = testDataRaw[testDataRaw['is_fraud'] == 1].sample(1)
print(random_fraud.shape)
random_fraud.head()

(1, 23)


,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
218302,218302,2020-09-07 00:28:46,3597926034019603,fraud_O'Keefe-Hudson,grocery_pos,258.85,Derrick,Flores,M,83690 Nicholas Ports Apt. 846,...,35.2229,-89.5518,9496,Furniture conservator/restorer,1993-03-23,df8f431aec59a1c81f6c0ea942f665e7,1378513726,34.886426,-89.282649,1


In [280]:
categories = testDataRaw['category'].unique()
print(categories)

['personal_care' 'health_fitness' 'misc_pos' 'travel' 'kids_pets'
 'shopping_pos' 'food_dining' 'home' 'entertainment' 'shopping_net'
 'misc_net' 'grocery_pos' 'gas_transport' 'grocery_net']


In [281]:
# Aplicar preprocesamiento
columns_out = ['Unnamed: 0', 'cc_num', 'merchant', 'state', 'first', 'last', 'gender', 'street', 'city', 'zip', 'city', 'job', 'dob', 'trans_num', 'unix_time', 'is_fraud']
random_fraud = random_fraud.drop(columns=columns_out)

In [282]:
# Category encoding
categories = ['personal_care', 'health_fitness', 'misc_pos', 'travel', 'kids_pets', 'shopping_pos', 'food_dining', 'home', 'entertainment', 'shopping_net', 'misc_net', 'grocery_pos', 'gas_transport', 'grocery_net']
for category in categories:
    random_fraud[f'cat_{category}'] = False

In [283]:
random_fraud.head()

,trans_date_trans_time,category,amt,lat,long,city_pop,merch_lat,merch_long,cat_personal_care,cat_health_fitness,...,cat_kids_pets,cat_shopping_pos,cat_food_dining,cat_home,cat_entertainment,cat_shopping_net,cat_misc_net,cat_grocery_pos,cat_gas_transport,cat_grocery_net
218302,2020-09-07 00:28:46,grocery_pos,258.85,35.2229,-89.5518,9496,34.886426,-89.282649,False,False,...,False,False,False,False,False,False,False,False,False,False


In [284]:
index = random_fraud.index[0]   
categ = random_fraud['category'][index]
cat = 'cat_' + str(categ)
print(cat)

cat_grocery_pos


In [285]:
print(random_fraud.columns)

Index(['trans_date_trans_time', 'category', 'amt', 'lat', 'long', 'city_pop',
       'merch_lat', 'merch_long', 'cat_personal_care', 'cat_health_fitness',
       'cat_misc_pos', 'cat_travel', 'cat_kids_pets', 'cat_shopping_pos',
       'cat_food_dining', 'cat_home', 'cat_entertainment', 'cat_shopping_net',
       'cat_misc_net', 'cat_grocery_pos', 'cat_gas_transport',
       'cat_grocery_net'],
      dtype='object')


In [286]:

# Asegúrate de que la columna exista
if cat not in random_fraud.columns:
    random_fraud[cat] = False

# Establecer la categoría a True
random_fraud.at[index, cat] = True



In [287]:
print(random_fraud.head())

       trans_date_trans_time     category     amt      lat     long  city_pop  \
218302   2020-09-07 00:28:46  grocery_pos  258.85  35.2229 -89.5518      9496   

        merch_lat  merch_long  cat_personal_care  cat_health_fitness  ...  \
218302  34.886426  -89.282649              False               False  ...   

        cat_kids_pets  cat_shopping_pos  cat_food_dining  cat_home  \
218302          False             False            False     False   

        cat_entertainment  cat_shopping_net  cat_misc_net  cat_grocery_pos  \
218302              False             False         False             True   

        cat_gas_transport  cat_grocery_net  
218302              False            False  

[1 rows x 22 columns]


In [288]:
# Date encoding
random_fraud['hour'] = pd.to_datetime(random_fraud['trans_date_trans_time']).dt.hour
random_fraud['day'] = pd.to_datetime(random_fraud['trans_date_trans_time']).dt.day_of_week
random_fraud['month'] = pd.to_datetime(random_fraud['trans_date_trans_time']).dt.month
random_fraud = random_fraud.drop(columns=['trans_date_trans_time'])
random_fraud = random_fraud.drop(columns=['category'])

In [289]:
random_fraud.head()

,amt,lat,long,city_pop,merch_lat,merch_long,cat_personal_care,cat_health_fitness,cat_misc_pos,cat_travel,...,cat_home,cat_entertainment,cat_shopping_net,cat_misc_net,cat_grocery_pos,cat_gas_transport,cat_grocery_net,hour,day,month
218302,258.85,35.2229,-89.5518,9496,34.886426,-89.282649,False,False,False,False,...,False,False,False,False,True,False,False,0,0,9


In [291]:
# Escalar los datos
scaler = StandardScaler()
random_line_scaled = scaler.fit_transform(random_fraud)

In [297]:
# Realizar la predicción
prediction = model.predict(random_line_scaled)
prediction_result = (prediction > 0.5)

print("Resultado de la predicción:", prediction_result)


1/1 [==============================] - 0s 22ms/step
Resultado de la predicción: [[False]]


# DATASET

In [298]:
# test
testDataRaw = pd.read_csv('../../../datasets/fraudTest.csv')
print(testDataRaw.shape)
print(testDataRaw.head())

(555719, 23)
   Unnamed: 0 trans_date_trans_time            cc_num  \
0           0   2020-06-21 12:14:25  2291163933867244   
1           1   2020-06-21 12:14:33  3573030041201292   
2           2   2020-06-21 12:14:53  3598215285024754   
3           3   2020-06-21 12:15:15  3591919803438423   
4           4   2020-06-21 12:15:17  3526826139003047   

                               merchant        category    amt   first  \
0                 fraud_Kirlin and Sons   personal_care   2.86    Jeff   
1                  fraud_Sporer-Keebler   personal_care  29.84  Joanne   
2  fraud_Swaniawski, Nitzsche and Welch  health_fitness  41.28  Ashley   
3                     fraud_Haley Group        misc_pos  60.05   Brian   
4                 fraud_Johnston-Casper          travel   3.19  Nathan   

       last gender                       street  ...      lat      long  \
0   Elliott      M            351 Darlene Green  ...  33.9659  -80.9355   
1  Williams      F             3638 Marsh Union  

In [299]:
columns_out = ['Unnamed: 0', 'cc_num', 'merchant', 'state', 'first', 'last', 'gender', 'street', 'city', 'zip', 'city', 'job', 'dob', 'trans_num', 'unix_time']
fraud_data = testDataRaw.drop(columns=columns_out)

# Category encoding
fraud_data = pd.get_dummies(fraud_data, columns=['category'], prefix='cat')

# Date encoding
fraud_data['hour'] = pd.to_datetime(fraud_data['trans_date_trans_time']).dt.hour
fraud_data['day'] = pd.to_datetime(fraud_data['trans_date_trans_time']).dt.day_of_week
fraud_data['month'] = pd.to_datetime(fraud_data['trans_date_trans_time']).dt.month
fraud_data = fraud_data.drop(columns=['trans_date_trans_time'])


In [303]:
fraud_data.head()

,amt,lat,long,city_pop,merch_lat,merch_long,is_fraud,cat_entertainment,cat_food_dining,cat_gas_transport,...,cat_kids_pets,cat_misc_net,cat_misc_pos,cat_personal_care,cat_shopping_net,cat_shopping_pos,cat_travel,hour,day,month
0,2.86,33.9659,-80.9355,333497,33.986391,-81.200714,0,False,False,False,...,False,False,False,True,False,False,False,12,6,6
1,29.84,40.3207,-110.4360,302,39.450498,-109.960431,0,False,False,False,...,False,False,False,True,False,False,False,12,6,6
2,41.28,40.6729,-73.5365,34496,40.495810,-74.196111,0,False,False,False,...,False,False,False,False,False,False,False,12,6,6
3,60.05,28.5697,-80.8191,54767,28.812398,-80.883061,0,False,False,False,...,False,False,True,False,False,False,False,12,6,6
4,3.19,44.2529,-85.0170,1126,44.959148,-85.884734,0,False,False,False,...,False,False,False,False,False,False,True,12,6,6


In [304]:
fraud_data = fraud_data.drop(columns=['is_fraud'])

In [305]:
loaded_model = tf.keras.models.load_model("detectorV6.h5")

In [306]:
scaler = StandardScaler()
fraud_data_scaled = scaler.fit_transform(fraud_data)

In [309]:
testData = testDataRaw.drop(columns=columns_out)

In [316]:
# Realizar la predicción
predictions = loaded_model.predict(fraud_data_scaled)
predictions = (predictions > 0.5)  # Convertir a True/False


17367/17367 [==============================] - 17s 960us/step


In [318]:
# Agregar las predicciones al DataFrame
testData['prediction'] = predictions.astype(int)

# Visualizar el DataFrame con las predicciones
print(testData[['is_fraud', 'prediction']])

        is_fraud  prediction
0              0           0
1              0           0
2              0           0
3              0           0
4              0           0
...          ...         ...
555714         0           0
555715         0           0
555716         0           0
555717         0           0
555718         0           0

[555719 rows x 2 columns]


In [320]:
only_fraud_rows = testData[(testData['is_fraud'] == 1)]
print(only_fraud_rows)

       trans_date_trans_time        category      amt      lat      long  \
1685     2020-06-21 22:06:39  health_fitness    24.84  31.8599 -102.7413   
1767     2020-06-21 22:32:22        misc_net   780.52  42.5545  -90.3508   
1781     2020-06-21 22:37:27   entertainment   620.33  42.5545  -90.3508   
1784     2020-06-21 22:38:55    shopping_net  1077.69  30.4590  -90.9027   
1857     2020-06-21 23:02:16    shopping_pos   842.65  31.8599 -102.7413   
...                      ...             ...      ...      ...       ...   
517197   2020-12-22 22:05:48    shopping_pos  1041.51  34.6323  -89.8855   
517274   2020-12-22 22:18:07    shopping_pos   868.09  34.6323  -89.8855   
517341   2020-12-22 22:31:48    shopping_net  1039.42  34.6323  -89.8855   
517529   2020-12-22 23:06:03     grocery_pos   289.27  34.6323  -89.8855   
517571   2020-12-22 23:13:39        misc_net   766.38  34.6323  -89.8855   

        city_pop  merch_lat  merch_long  is_fraud  prediction  
1685          23  32.57

In [319]:
fraud_rows = testData[(testData['is_fraud'] == 1) & (testData['prediction'] == 1)]
print("Filas donde is_fraud y prediction son iguales a 1:")
print(fraud_rows)

Filas donde is_fraud y prediction son iguales a 1:
       trans_date_trans_time      category      amt      lat      long  \
1767     2020-06-21 22:32:22      misc_net   780.52  42.5545  -90.3508   
1784     2020-06-21 22:38:55  shopping_net  1077.69  30.4590  -90.9027   
1857     2020-06-21 23:02:16  shopping_pos   842.65  31.8599 -102.7413   
1906     2020-06-21 23:21:44  shopping_net  1128.26  30.4590  -90.9027   
1956     2020-06-21 23:35:27  shopping_net   931.82  30.4590  -90.9027   
...                      ...           ...      ...      ...       ...   
511675   2020-12-21 23:57:16  shopping_net   981.22  34.6323  -89.8855   
514913   2020-12-22 14:30:10  shopping_net   997.77  34.6323  -89.8855   
517341   2020-12-22 22:31:48  shopping_net  1039.42  34.6323  -89.8855   
517529   2020-12-22 23:06:03   grocery_pos   289.27  34.6323  -89.8855   
517571   2020-12-22 23:13:39      misc_net   766.38  34.6323  -89.8855   

        city_pop  merch_lat  merch_long  is_fraud  predictio